In [1]:
import os
from lxml import etree
import time

import sys
import os

sys.path.append("/home/parvez/PycharmProjects/MigrationRecommendation/src")

from Utility.SiteManager import SiteManager
from xmlreader.DataConverter import DataConverter
import numpy as np
import pandas as pd
STACK_EXCHANGE_DATA = "/media/parvez/SamsungOneTB/MigrationRecom/StackExchangeData"

list_subfolders_with_paths = [(f.path,f.name) for f in os.scandir(STACK_EXCHANGE_DATA) if f.is_dir()]
for path in list_subfolders_with_paths:
    print(path)

site_manager = SiteManager("/media/parvez/SamsungOneTB/MigrationRecom/data/sites_category.csv")

stackoverflow.com
cogsci.stackexchange.com
dba.meta.stackexchange.com
('/media/parvez/SamsungOneTB/MigrationRecom/StackExchangeData/latin.meta.stackexchange.com', 'latin.meta.stackexchange.com')
('/media/parvez/SamsungOneTB/MigrationRecom/StackExchangeData/music.stackexchange.com', 'music.stackexchange.com')
('/media/parvez/SamsungOneTB/MigrationRecom/StackExchangeData/vegetarianism.stackexchange.com', 'vegetarianism.stackexchange.com')
('/media/parvez/SamsungOneTB/MigrationRecom/StackExchangeData/startups.stackexchange.com', 'startups.stackexchange.com')
('/media/parvez/SamsungOneTB/MigrationRecom/StackExchangeData/astronomy.meta.stackexchange.com', 'astronomy.meta.stackexchange.com')
('/media/parvez/SamsungOneTB/MigrationRecom/StackExchangeData/meta.pt.stackoverflow.com', 'meta.pt.stackoverflow.com')
('/media/parvez/SamsungOneTB/MigrationRecom/StackExchangeData/softwareengineering.stackexchange.com', 'softwareengineering.stackexchange.com')
('/media/parvez/SamsungOneTB/MigrationRecom

In [2]:
#load the user data
start_time = time.time()
siteToUserData = {}
site_count = 0
for (path,name) in list_subfolders_with_paths:
    count = 0
    userIdToUserDict = {}
    for event, elem in etree.iterparse(path+"/Users.xml", events=("start","end","start-ns","end-ns")):
        if elem.tag == "row" and event == "start":
            count = count + 1
            if(count%1000000==0):
                print("Progress of reading users: "+str(count))
            lu = DataConverter.readLightUser(elem)
            userIdToUserDict[lu.get_id()] = lu
    siteToUserData[name] = userIdToUserDict
    print("Completed: "+str(site_count)+"/"+str(len(list_subfolders_with_paths)))
    site_count = site_count + 1
print("--- %s seconds ---" % (time.time() - start_time))

Completed: 0/350
Completed: 1/350
Completed: 2/350
Completed: 3/350
Completed: 4/350
Completed: 5/350


KeyboardInterrupt: 

In [3]:
#load the post history data
start_time = time.time()
siteToPostHistoryDict = {}
site_count = 0
for (path,name) in list_subfolders_with_paths:
    count = 0
    phIdToPhDict = {}
    for event, elem in etree.iterparse(path+"/PostHistory.xml", events=("start","end","start-ns","end-ns")):
        if elem.tag == "row" and event == "start":
            count = count + 1
            if(count%1000000==0):
                print("Progress of reading post histories: "+str(count))
            ph = DataConverter.readPostHistory(elem)
            ph.set_site(site_manager.getSiteName(name))

            if ph.get_postHistoryTypeId()==35:
                site_name = site_manager.getSiteName(ph.get_destination())
                ph.set_destination(site_name)
                if ph.get_destination() is not None and ph.get_site() is not None:
                    phIdToPhDict[ph.get_id()] = ph
                    ph.print_postHistory()
            elif ph.get_postHistoryTypeId()==36:
                site_name = site_manager.getSiteName(ph.get_origin())
                ph.set_origin(site_name)
                if ph.get_origin() is not None and ph.get_site() is not None:
                    phIdToPhDict[ph.get_id()] = ph
                    ph.print_postHistory()

    siteToPostHistoryDict[name] = phIdToPhDict
    print("Completed: "+str(site_count)+"/"+str(len(list_subfolders_with_paths)))
    site_count = site_count + 1
print("--- %s seconds ---" % (time.time() - start_time))

TypeError: readPostHistory() missing 1 required positional argument: 'site_manager'

In [ ]:
import pickle
#Writing the data to a csv file
columns = ['id','post_type_id','parent_id','creation_date','score','view_count','owner_user_id',"file_position"]
data_list = [[],[],[],[],[],[],[],[]]
start_time = time.time()
site_count = 0
siteNameToPostDict = {}
with open("/media/parvez/SamsungOneTB/MigrationRecom/serialize/posts.ser","wb") as serializePostFile:
    for (path,name) in list_subfolders_with_paths:
        count = 0
        for event, elem in etree.iterparse(path+"/Posts.xml", events=("start","end","start-ns","end-ns")):
            site_id = site_manager.nameToIdDict[name]
            site_category =  site_manager.idToCategoryDict[site_id]
            if elem.tag == "row" and event == "start" and site_category==1:
                count = count + 1
                if(count%1000000==0):
                    print("Progress of reading posts: "+str(count))
                post = DataConverter.readPost(elem)
                start_position = serializePostFile.tell()
                pickle.dump(obj=post,file=serializePostFile)
                data_list[0].append(post.get_id())
                data_list[1].append(post.get_postTypeId())
                data_list[2].append(post.get_parentId())
                data_list[3].append(post.get_creationDate())
                data_list[4].append(post.get_score())
                data_list[5].append(post.get_viewCount())
                data_list[6].append(post.get_ownerUserId())
                data_list[7].append(start_position)

        df_post = pd.DataFrame({columns[0]:data_list[0],columns[1]:data_list[1],columns[2]:data_list[2],columns[3]:data_list[3],columns[4]:data_list[4],
                             columns[5]:data_list[5],columns[6]:data_list[6],columns[7]:data_list[7]})

        siteNameToPostDict[path] = df_post
        print("Completed: "+str(site_count)+"/"+str(len(list_subfolders_with_paths)))
        site_count = site_count + 1
with open("/media/parvez/SamsungOneTB/MigrationRecom/serialize/post_dict.ser","wb") as serializePostDictFile:
    pickle.dump(obj=siteNameToPostDict,file=serializePostDictFile)
print("Time required to read posts: "+str(time.time()-start_time))

Completed: 0/350
Completed: 1/350
Completed: 2/350
Completed: 3/350
Completed: 4/350
Completed: 5/350
Completed: 6/350
Completed: 7/350
Completed: 8/350
Completed: 9/350
Completed: 10/350
Completed: 11/350
Completed: 12/350
Completed: 13/350
Completed: 14/350
Completed: 15/350
Completed: 16/350
Completed: 17/350
Completed: 18/350
Completed: 19/350
Completed: 20/350
Completed: 21/350
Completed: 22/350
Completed: 23/350
Completed: 24/350
Completed: 25/350
Completed: 26/350
Completed: 27/350
Completed: 28/350
Completed: 29/350
Completed: 30/350
Completed: 31/350
Completed: 32/350
Completed: 33/350
Completed: 34/350
Completed: 35/350
Completed: 36/350
Completed: 37/350
Completed: 38/350
Completed: 39/350
Completed: 40/350
Completed: 41/350
Completed: 42/350
Completed: 43/350
Completed: 44/350
Completed: 45/350
Completed: 46/350
Completed: 47/350
Completed: 48/350
Completed: 49/350
Completed: 50/350
Completed: 51/350
Completed: 52/350
Completed: 53/350
Completed: 54/350
Completed: 55/350
Co

In [ ]:
columns = ['id','reputation','creation_date','display_name','views','upvotes','downvotes','account_id']
data_list = [[],[],[],[],[],[],[],[]]
start_time = time.time()
count = 0
dataframe = pd.DataFrame(columns = columns)
for (path,name) in list_subfolders_with_paths:
    for event, elem in etree.iterparse("/Users.xml", events=("start","end","start-ns","end-ns")):
        if elem.tag == "row" and event == "start":
            count = count + 1
            if(count%1000000==0):
                print("Progress of reading users: "+str(count))
            user = DataConverter.readUser(elem)
            data_list[0].append(user.get_id())
            data_list[1].append(user.get_reputation())
            data_list[2].append(user.get_creationDate())
            data_list[3].append(user.get_displayName())
            data_list[4].append(user.get_views())
            data_list[5].append(user.get_upVotes())
            data_list[6].append(user.get_downVotes())
            data_list[7].append(user.get_accountId())

df_user = pd.DataFrame({columns[0]:data_list[0],columns[1]:data_list[1],columns[2]:data_list[2],columns[3]:data_list[3],columns[4]:data_list[4],
                         columns[5]:data_list[5],columns[6]:data_list[6],columns[7]:data_list[7]})
print("Time required to read users: "+str(time.time()-start_time))


columns = ['id','post_id','vote_type_id','user_id','creation_date','bounty_amount']
data_list = [[],[],[],[],[],[]]
start_time = time.time()
count = 0
for (path,name) in list_subfolders_with_paths:
    for event, elem in etree.iterparse("/Votes.xml", events=("start","end","start-ns","end-ns")):
        if elem.tag == "row" and event == "start":
            count = count + 1
            if(count%1000000==0):
                print("Progress of reading votes: "+str(count))
            vote = DataConverter.readVote(elem)
            data_list[0].append(vote.get_id())
            data_list[1].append(vote.get_postId())
            data_list[2].append(vote.get_voteTypeId())
            data_list[3].append(vote.get_userId())
            data_list[4].append(vote.get_creationDate())
            data_list[5].append(vote.get_bountyAmount())

df_vote = pd.DataFrame({columns[0]:data_list[0],columns[1]:data_list[1],columns[2]:data_list[2],columns[3]:data_list[3],columns[4]:data_list[4],
                         columns[5]:data_list[5]})

In [ ]:
columns = ['id','ph_type_id','post_id','revision_guid','creation_date','user_id','user_display_name','comment','text','destination','destination_post_id','origin','origin_post_id']
data_list = [[],[],[],[],[],[],[],[],[],[],[],[],[]]
start_time = time.time()

site_count = 0
dataframe = pd.DataFrame(columns = columns)
for (path,name) in list_subfolders_with_paths:
    count = 0
    print("Processsing site: "+name)
    for event, elem in etree.iterparse(path+"/PostHistory.xml", events=("start","end","start-ns","end-ns")):
        if elem.tag == "row" and event == "start":
            count = count + 1
            if(count%1000000==0):
                print("Progress of reading post histories: "+str(count))
            postHistory = DataConverter.readPostHistory(elem,site_manager)
            if postHistory.get_postHistoryTypeId()==35 or postHistory.get_postHistoryTypeId()==36:
                data_list[0].append(postHistory.get_id())
                data_list[1].append(postHistory.get_postHistoryTypeId())
                data_list[2].append(postHistory.get_postId())
                data_list[3].append(postHistory.get_revisionGUID())
                data_list[4].append(postHistory.get_creationDate())
                data_list[5].append(postHistory.get_userId())
                data_list[6].append(postHistory.get_userDisplayName())
                data_list[7].append(postHistory.get_comment())
                data_list[8].append(postHistory.get_text())
                data_list[9].append(postHistory.get_destination())
                data_list[10].append(postHistory.get_destinationId())
                data_list[11].append(postHistory.get_origin())
                data_list[12].append(postHistory.get_originId())
    print("Completed: "+str(site_count)+"/"+str(len(list_subfolders_with_paths)))
    site_count = site_count + 1
df_ph = pd.DataFrame({columns[0]:data_list[0],columns[1]:data_list[1],columns[2]:data_list[2],columns[3]:data_list[3],columns[4]:data_list[4],columns[5]:data_list[5],columns[6]:data_list[6],columns[7]:data_list[7],columns[8]:data_list[8]})
print("Time required to read post histories: "+str(time.time()-start_time))


Processsing site: latin.meta.stackexchange.com
Completed: 0/350
Processsing site: music.stackexchange.com
Completed: 1/350
Processsing site: vegetarianism.stackexchange.com
Completed: 2/350
Processsing site: startups.stackexchange.com
Completed: 3/350
Processsing site: astronomy.meta.stackexchange.com
Completed: 4/350
Processsing site: meta.pt.stackoverflow.com
Completed: 5/350
Processsing site: softwareengineering.stackexchange.com
Completed: 6/350
Processsing site: tridion.stackexchange.com
Completed: 7/350
Processsing site: german.meta.stackexchange.com
Completed: 8/350
Processsing site: quant.stackexchange.com
Completed: 9/350
Processsing site: productivity.stackexchange.com
Completed: 10/350
Processsing site: cs.stackexchange.com
Completed: 11/350
Processsing site: iota.meta.stackexchange.com
Completed: 12/350
Processsing site: physics.stackexchange.com
Cannot find: firearms.stackexchange.com
Cannot find: firearms.stackexchange.com
Cannot find: firearms.stackexchange.com
Progress 